In [1]:
'''
This handles logging in to Spotify and returning a Spotipy object
'''

import os

import spotipy
import spotipy.util as util
import pandas as pd

import config

username = config.CLIENT_USERNAME


In [2]:
def login_to_spotify():
    '''
    handles logging in to spotify and returning a spotipy object
    '''

    os.environ['SPOTIPY_CLIENT_ID'] = config.CLIENT_ID
    os.environ['SPOTIPY_CLIENT_SECRET'] = config.CLIENT_SECRET
    os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:8888/callback/'

    username = ''

    scope = 'user-library-read playlist-read-private user-top-read'

    token = util.prompt_for_user_token(
        username,
        scope
    )

    spotipy_wrapper = spotipy.Spotify(
        auth=token
    )

    if token:
        spotipy_wrapper = spotipy.Spotify(auth=token)
        return spotipy_wrapper

    print("Can't get token for", username)
    return None

In [3]:
SPOTIPY_OBJECT = login_to_spotify()
playlists = SPOTIPY_OBJECT.user_playlists(username)

In [4]:
for playlist in playlists['items']:
    if (playlist['owner']['id'] == username) and \
        (playlist['name'] == 'likes'):
        likes_id = playlist['id']
    elif (playlist['owner']['id'] == username) and \
        (playlist['name'] == 'dislikes'):
        dislikes_id = playlist['id']
    elif (playlist['owner']['id'] == username) and \
        (playlist['name'] == 'amb-tech'):
        amb_tech_id = playlist['id']

results = SPOTIPY_OBJECT.user_playlist(
                username,
                amb_tech_id,
            )['tracks']['items']

In [5]:
track_ids = []

for track in results:
     track_ids.append(track['track']['id'])
    
# track_ids

['3yWl69NlEkqdGDUtLzDQl4',
 '5zSGNCPUCQXsqzdYAggfUe',
 '5RSLE68Xq2m6h7ggtVReSL',
 '50PtCIy3QV8JTfmXISe2I7',
 '2EAwwq6Rq2KOE6sgth13ah',
 '0WsROU8CJrMWBukK5IMs4y',
 '6JfhpeR95WeRE3xnXB1pPo',
 '1H5neWFEXm8YnlXGH3MhIW',
 '2Cq5ETe5Wit7ZMV7N1Ogxe',
 '7gECnmpNOXBRinZCsTrDKm',
 '21DNvUd4vvPlytmPh9nIXA',
 '4TQmCn0e9L7XTiKY4Hk8X4',
 '2ZAQ5A65YqpQ7vLiRXebSE']

In [6]:
track_features_list = SPOTIPY_OBJECT.audio_features(tracks=track_ids)
track_features = {}

for i in range(len(track_ids)):
    track_features[track_ids[i]] = track_features_list[i]


In [7]:
track_analysis = {}

artist_ids = {}
artist_genres = {}
artist_names = {}
artist_popularity = {}
artist_followers = {}

for track_id in track_ids:
    track_analysis[track_id] = SPOTIPY_OBJECT.audio_analysis(track_id)['track']
    
    artist_id = SPOTIPY_OBJECT.track(track_id)['artists'][0]['id']
    artist_ids[track_id] = artist_id
    
    artist_info = SPOTIPY_OBJECT.artist(artist_id)
    
    artist_genres[artist_id] = ','.join(artist_info['genres'])
    artist_names[artist_id] = artist_info['name']
    artist_popularity[artist_id] = artist_info['popularity']
    artist_followers[artist_id] = artist_info['followers']['total']
    

In [8]:
artist_ids_df = pd.DataFrame.from_dict(
    artist_ids,
    orient='index',
    columns=['artist_id']
).reset_index().rename(columns={'index': 'track_id'}).astype({'track_id': str, 'artist_id': str})

artist_names_df = pd.DataFrame.from_dict(
    artist_names,
    orient='index',
    columns=['artist_name']
).reset_index().rename(columns={'index': 'artist_id'}).astype({'artist_id': str, 'artist_name': str})

artist_popularity_df = pd.DataFrame.from_dict(
    artist_popularity,
    orient='index',
    columns=['artist_popularity']
).reset_index().rename(columns={'index': 'artist_id'}).astype({'artist_id': str, 'artist_popularity': int})

artist_followers_df = pd.DataFrame.from_dict(
    artist_followers,
    orient='index',
    columns=['artist_followers']
).reset_index().rename(columns={'index': 'artist_id'}).astype({'artist_id': str, 'artist_followers': int})

artist_genres_df = pd.DataFrame.from_dict(
    artist_genres,
    orient='index',
    columns=['artist_genres']
).reset_index().rename(columns={'index': 'artist_id'}).astype({'artist_id': str, 'artist_genres': str})

track_features_df = pd.DataFrame.from_dict(
    track_features,
    orient='index'
).reset_index().rename(columns={'id': 'track_id'}).drop(columns=['index'])


track_analysis_df = pd.DataFrame.from_dict(
    track_analysis,
    orient='index'
).reset_index().drop(
    columns=[
        'codestring',
        'rhythmstring',
        'synchstring',
        'code_version',
        'echoprint_version',
        'rhythm_version',
        'synch_version',
    ]
).rename(columns={'index': 'track_id'})
